<a href="https://colab.research.google.com/github/renatoponchi/raizen/blob/main/raizen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from pyspark.sql.types import *
import pyspark
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import *

path = "/content/drive/MyDrive/2000.csv"

df = spark.createDataFrame(pd.read_csv(path, sep=';'))\
.withColumnRenamed("COMBUSTÍVEL","combustivel")\
.withColumnRenamed("REGIÃO","regiao")\
.drop("TOTAL")

def melt(df: DataFrame,id_vars: Iterable[str], value_vars: Iterable[str],var_name: str="variable", value_name: str="value") -> DataFrame:
    _vars_and_vals = array(*(struct(lit(c).alias(var_name), col(c).alias(value_name)) for c in value_vars))
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))
    cols = id_vars + [col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

df_ = melt(df,
     id_vars=['combustivel','ANO','regiao','ESTADO','UNIDADE'],
     value_vars=['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez'])

df_.show()

NameError: ignored